In [1]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
#from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w1 = torch.from_numpy(np.random.normal(0, 1,(num_X1feature,1)))
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w1:',true_w1)
        print('true_w2:',true_w2)
    X1_ = 5*torch.from_numpy(np.random.normal(0, 1, (num_example, num_X1feature)))
    X2_before_ =5* torch.from_numpy(np.random.normal(0, 1, (num_example, num_X2feature)))
    y_ = torch.mm(X1_,true_w1)+torch.mm(X2_before_,true_w2)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.mm(X1_,true_w1)-torch.mm(X2_before_,true_w2))/torch.norm(y_-torch.mm(X1_,true_w1)))
    return y_,X1_,X2_,true_w1,true_w2,P,error_reg

In [3]:
def SinkhornIPOT(X,eps=5e-3,beta=12,num_L=1):
    l=torch.ones(num_example,1,dtype=torch.float64)
    b=l/num_example
    G=torch.exp(-X/beta)
    gama=torch.ones(num_example,num_example,dtype=torch.float64)   
    i=0
    before=torch.zeros(num_example,num_example,dtype=torch.float64)  
    while True:
        Q=G*gama
        for j in range(num_L):
            a=l/(torch.mm(Q,b))
            b=l/(torch.mm(Q.transpose(1,0),a))
        gama=torch.mm(torch.mm(torch.diag(a.squeeze(1)),Q),torch.diag(b.squeeze(1)))
        if i%20==0:
            if torch.norm(gama-before)<eps:
                break
            before=gama            
            #print('IPOT循环是否收敛',torch.norm(ga2-ga1))
        i+=1
        if i>=100000:
            break
        if math.isnan(gama[0][0]):
            break
    #print(gama.half())
    #print(i)
    return gama

In [4]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w1 = torch.from_numpy(np.random.normal(0, 1,(num_X1feature,1)))
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w1=torch.zeros(num_X1feature,1,dtype=torch.float64)
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if method=='regression':
        X=torch.cat([X1,X2],1)
        w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
        w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    if showinitialw:
        print('initial w1:',w1)
        print('initial w2:',w2)
    return w1,w2

In [5]:
gama_=3
eta=0
starts=1
for num_example in range(60,91,10):
    #for num_X2feature in range(2,6,1):    
    for num_X1feature in range(1,6,1):
        num_X2feature=num_X1feature
        for i____ in range(2):
            #num_epochs=70
            (y_,X1_,X2_,true_w1,true_w2,true_P,error_reg1)=generatedata(noise=0.5)
            y=y_
            X1=X1_
            X2=X2_
            results_Loss = []
            results_w1=[]
            results_w2=[]
            results_error=[]
            for i__ in range(starts):
#                     P_array=np.random.permutation(num_example)
#                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
#                     for i in range(num_example):
#                         P[i][P_array[i]]=1
#                     X_=torch.cat([X1,X2],1)
#                     X=torch.mm(P,X_)
#                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
#                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
#                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
#                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                w1,w2=generateinitialw(method='zeros')
                #w1=true_w1
                #w2=true_w2
                w1.requires_grad_(requires_grad=True)
                w2.requires_grad_(requires_grad=True)
#                 results_Loss = []
                lr=0.0002
                #results_S=[]
                t=0
                before1=0
                while True:
                #for t in range(num_epochs):                        

                    Y1=y-torch.mm(X1,w1)
                    Y2=torch.mm(X2,w2)
                    C=torch.zeros(num_example,num_example,dtype=torch.float64)
                    for i in range(num_example):
                        for j in range(num_example):
                            C[i][j]=(Y1[i]-Y2[j])**2            
 
                    S=SinkhornIPOT(C)
                    #print(S.transpose(1,0).half())
                    #results_S.append(S)
                    #if t>0:
                        #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                    #Loss=torch.sum(S*C)
                    Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                    if Loss<3e-2:
                        break
                    Loss.backward()
#                         results_Loss.append(Loss)
#                         for i_ in range(num_X1features):
#                             results_w1[t][i_]=(w1[i_].data)
#                         for i_ in range(num_X2features):
#                             results_w2[t][i_]=(w2[i_].data)
                    w1.data-=lr*(w1.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                    w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                    #print(w1.grad)
                    #print(w2.grad)
#                     if t==num_epochs-1:
#                         print('最终w1梯度：',w1.grad)
#                         print('最终w2梯度：',w2.grad)
                    w1.grad.data.zero_() 
                    w2.grad.data.zero_() 

                    #print('Loss',t,'=',Loss)
                    if t%6==0:
                        if torch.norm(Loss-before1)<1e-4:
                            break
                        before1=Loss                                  
                    if t>=500:
                        break
                    if math.isnan(Loss):
                        break
                    t+=1
                    print('#',end='')
                    


                print(' ',end='')
                error_each=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                results_error.append(error_each)
                #results_Loss.append(Loss)
                #results_w1.append(w1.data)
                #results_w2.append(w2.data)


            #w1=results_w1[results_Loss.index(min(results_Loss))]
            #w2=results_w2[results_Loss.index(min(results_Loss))]

#                     for i_ in range(starts):
#                         results_w1[i_]=(w1[i_].data)
#                     for i_ in range(starts):
#                         results_w2[i_]=(w2[i_].data)


            #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
            #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
            print(num_X1feature,num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')
            
            #print('真实置换矩阵为：',true_P)
            error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
            print(num_X1feature,num_X2feature,num_example,'置换矩阵误差：',error_P,end='   ')
            error_reg2=(torch.norm(y_-torch.mm(X1_,w1)-torch.mm(torch.mm(S,X2_),w2))/torch.norm(y_-torch.mm(X1_,w1)))
            print('真实回归误差',error_reg1,end='   ')
            print('实验回归误差',error_reg2)
            #print('双随机矩阵S为：',S.transpose(1,0).half())
            #print(results)
#                 plt.figure(figsize=(6,6))
#                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
#                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
#                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
#                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
#                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
#                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
#                 plt.legend()
#                 plt.show()

######################## 1 1 60 平均相对误差2： tensor(0.0727, dtype=torch.float64, grad_fn=<DivBackward0>)   1 1 60 置换矩阵误差： tensor(0.9916, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0530, dtype=torch.float64)   实验回归误差 tensor(0.0343, dtype=torch.float64, grad_fn=<DivBackward0>)
###################################################### 1 1 60 平均相对误差2： tensor(0.3246, dtype=torch.float64, grad_fn=<DivBackward0>)   1 1 60 置换矩阵误差： tensor(1.1418, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.8445, dtype=torch.float64)   实验回归误差 tensor(0.1446, dtype=torch.float64, grad_fn=<DivBackward0>)
###################################################################################################### 2 2 60 平均相对误差2： tensor(0.1377, dtype=torch.float64, grad_fn=<DivBackward0>)   2 2 60 置换矩阵误差： tensor(1.2277, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0842, dtype=torch.float64)   实验回归误差 tensor(0.0716, dtype=torch.float64, grad_fn=<DivBackward0>)
################

KeyboardInterrupt: 